# Setup

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


##Install required libraries

In [ ]:
!pip3 install 'torch'
!pip3 install 'torchvision'
!pip3 install 'pillow'
!pip3 install 'tqdm'
!pip3 install 'matplotlib'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Clone GitHub repository 

In [ ]:
import os

if not os.path.isdir('fedmd'):
  !git clone https://github.com/sergiuabed/fedmd
  %cd fedmd/
  !git checkout experiment
  %cd ..
else:
  %cd fedmd/
  !git pull origin
  %cd ..

if not os.path.isdir('fedmd/baseline_public_cifar10'):
  !mkdir fedmd/baseline_public_cifar10

Cloning into 'fedmd'...
remote: Enumerating objects: 2157, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 2157 (delta 17), reused 24 (delta 12), pack-reused 2125
Receiving objects: 100% (2157/2157), 314.62 MiB | 25.76 MiB/s, done.
Resolving deltas: 100% (781/781), done.
/content/fedmd
Updating files: 100% (416/416), done.
Branch 'experiment' set up to track remote branch 'experiment' from 'origin'.
Switched to a new branch 'experiment'
/content


## Import libraries

In [ ]:
from fedmd.models_implementations.resnet import ResNet
from fedmd.models_implementations.densenet import DenseNet
from fedmd.models_implementations.shufflenetv2 import ShuffleNetV2
from fedmd.models_implementations.utils import model_size, plot_stats
from fedmd.models_implementations.train_on_cifar import train_on_cifar
from fedmd.data_utils import read_data_splits
from fedmd.client.private_dataloader import ClientPrivateDataset
from torch.utils.data import DataLoader
from fedmd.client.client import Client

## Define a function to read from file the training statistics

In [ ]:
def read_stats(file_path:str) -> tuple[list]:
    with open(file_path, "r") as f:
        f.readline()
        lines = f.readlines()
    values = [line.rstrip("\n").split(",") for line in lines ]
    stats_tuples = [(int(v[0]), float(v[1]), float(v[2])) for v in values ]
    epochs, loss, acc = [list(i) for i in zip(*stats_tuples)]
    return epochs, loss, acc

#Train ResNet20
In this notebook we train ResNet, DenseNet and ShuffleNet architectures in a centralized way, in order to obtain baselines for subsequent experiments.


## Train ResNet20 on Cifar10

In [ ]:
if not os.path.isdir('fedmd/baseline_public_cifar10/resnet20'):
  !mkdir fedmd/baseline_public_cifar10/resnet20

  LR = 1e-1     
  MOMENTUM = 0.9       
  WEIGHT_DECAY = 1e-4  
  NUM_EPOCHS = 160
  FILE_PATH =  os.getcwd() + '/fedmd/baseline_public_cifar10/resnet20'
  NUM_CLASSES = 10
  GROUPS = 2
  LAYERS = [3, 3, 3]

  #!mkdir fedmd/baseline_public_cifar10/resnet20/resnet20_bn/
  #net = ResNet(layers = LAYERS, num_classes = NUM_CLASSES)
  #print(f"Num Parameters = {model_size(net)}")
  #train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/resnet20_bn')


  !mkdir fedmd/baseline_public_cifar10/resnet20/resnet20_gn/
  net = ResNet(layers = LAYERS, num_classes = NUM_CLASSES, norm_layer="gn", groups=GROUPS)
  print(f"Num Parameters = {model_size(net)}")
  train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/resnet20_gn')

Num Parameters = 2372


100%|██████████| 170498071/170498071 [00:05<00:00, 29942715.70it/s]


Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified
Train Dataset: 40000
Valid Dataset: 10000
Test Dataset: 10000
Starting epoch 1/160, LR = 0.1
Current Avg Loss = 2.356976631360176
Current Val Accuracy = 0.24091796875
Starting epoch 2/160, LR = 0.1
Current Avg Loss = 1.9335980537610176
Current Val Accuracy = 0.33544921875
Starting epoch 3/160, LR = 0.1
Current Avg Loss = 1.8062691321739783
Current Val Accuracy = 0.38984375
Starting epoch 4/160, LR = 0.1
Current Avg Loss = 1.7329749082907653
Current Val Accuracy = 0.39140625
Starting epoch 5/160, LR = 0.1
Current Avg Loss = 1.6701652331229968
Current Val Accuracy = 0.44755859375
Starting epoch 6/160, LR = 0.1
Current Avg Loss = 1.6114219274276342
Current Val Accuracy = 0.455859375
Starting epoch 7/160, LR = 0.1
Current Avg Loss = 1.5786737295297475
Current Val Accuracy = 0.46943359375
Starting epoch 8/160, LR = 0.1
Current Avg Loss = 1.52775632418119
Current Val Accuracy = 0.48291015625
Starting

### Plot average loss and validation accuracy per epoch

In [ ]:
epochs = [i + 1 for i in range(NUM_EPOCHS)]
_, loss_bn, acc_bn = read_stats(FILE_PATH + '/resnet20_bn/stats.csv')
_, loss_gn, acc_gn = read_stats(FILE_PATH + '/resnet20_gn/stats.csv')

filename = FILE_PATH + "/resnet20_loss"
plot_stats(x=epochs, data=[loss_bn, loss_gn], save_path=filename, title="Loss per Epoch", x_label="Epoch", y_label="Loss", data_labels=["ResNet-20 BN", "ResNet-20 GN"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(loss_bn), max(loss_gn))))

acc_bn = [100*a for a in acc_bn]
acc_gn = [100*a for a in acc_gn]
filename = FILE_PATH + "/resnet20_accuracy"
plot_stats(x=epochs, data=[acc_bn, acc_gn], save_path=filename, title="Validation Accuracy per Epoch", x_label="Epoch", y_label="Accuracy [%]", data_labels=["ResNet-20 BN", "ResNet-20 GN"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(acc_bn), max(acc_gn))))


In [ ]:
zip_name = "/content/fedmd/baseline_public_cifar10_batch1024.zip"
!zip -r {zip_name} /content/fedmd/baseline_public_cifar10/
!cp {zip_name} /content/drive/MyDrive/understanding_federated_learning_project/baselines_results

## Train DenseNet20 and DenseNet10

In [ ]:
if not os.path.isdir('fedmd/baseline_public_cifar10/densenet'):
  !mkdir fedmd/baseline_public_cifar10/densenet

  K = 12
  LR = 1e-1     
  MOMENTUM = 0.9       
  WEIGHT_DECAY = 1e-4  
  NUM_EPOCHS = 100
  FILE_PATH =  os.getcwd() + '/fedmd/baseline_public_cifar10/densenet'
  NUM_CLASSES = 10
  GROUPS = 2

  !mkdir fedmd/baseline_public_cifar10/densenet/densenet20/
  net = DenseNet(K, 20, 1, NUM_CLASSES, False, GROUPS)
  print(f"Num Parameters = {model_size(net)}")
  train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/densenet20')

  !mkdir fedmd/baseline_public_cifar10/densenet/densenet10/
  net = DenseNet(K, 10, 1, NUM_CLASSES, False, GROUPS)
  print(f"Num Parameters = {model_size(net)}")
  train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/densenet10')



### Plot average loss and validation accuracy per epoch

In [ ]:
epochs = [i + 1 for i in range(NUM_EPOCHS)]
_, loss_dn20, acc_dn20 = read_stats(FILE_PATH + '/densenet20/stats.csv')
_, loss_dn10, acc_dn10 = read_stats(FILE_PATH + '/densenet10/stats.csv')

filename = FILE_PATH + "/densenet_loss"
plot_stats(x=epochs, data=[loss_dn20, loss_dn10], save_path=filename, title="Loss per Epoch", x_label="Epoch", y_label="Loss", data_labels=["DenseNet-20", "DenseNet-10"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(loss_dn20), max(loss_dn10))))

acc_dn20 = [100*a for a in acc_dn20]
acc_dn10 = [100*a for a in acc_dn10]
filename = FILE_PATH + "/densenet_accuracy"
plot_stats(x=epochs, data=[acc_dn20, acc_dn10], save_path=filename, title="Validation Accuracy per Epoch", x_label="Epoch", y_label="Accuracy [%]", data_labels=["DenseNet-20", "DenseNet-10"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(acc_dn10), max(acc_dn20))))




In [ ]:
zip_name = "/content/fedmd/baseline_public_cifar10_2.zip"
!zip -r {zip_name} /content/fedmd/baseline_public_cifar10/
!cp {zip_name} /content/drive/MyDrive/understanding_federated_learning_project/baselines_results

## Train ShuffleNetBig and ShuffleNetSmall

In [ ]:
if not os.path.isdir('fedmd/baseline_public_cifar10/shufflenetv2'):
  !mkdir fedmd/baseline_public_cifar10/shufflenetv2

  LR = 1e-1     
  MOMENTUM = 0.9       
  WEIGHT_DECAY = 1e-4  
  NUM_EPOCHS = 200
  FILE_PATH =  os.getcwd() + '/fedmd/baseline_public_cifar10/shufflenetv2'
  NUM_CLASSES = 10
  GROUPS = 2

#  !mkdir fedmd/baseline_public_cifar10/shufflenetv2/shufflenetbig/
#  stages_repeats = [4, 8, 4]
#  stages_out_channels = [32, 64, 128, 256, 512]
#  net = ShuffleNetV2(stages_repeats, stages_out_channels, NUM_CLASSES, GROUPS)
#  print(f"Num Parameters = {model_size(net)}")
#  train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/shufflenetbig')

  !mkdir fedmd/baseline_public_cifar10/shufflenetv2/shufflenetsmall/
  stages_repeats = [2, 4, 2]
  stages_out_channels = [16, 32, 64, 128, 256]
  net = ShuffleNetV2(stages_repeats, stages_out_channels, NUM_CLASSES, GROUPS)
  print(f"Num Parameters = {model_size(net)}")
  train_on_cifar(net, NUM_EPOCHS, LR, MOMENTUM, WEIGHT_DECAY, cifar_100=False, file_path=FILE_PATH + '/shufflenetsmall')



Num Parameters = 4100


100%|██████████| 170498071/170498071 [00:12<00:00, 13240735.33it/s]


Extracting /content/cifar-10-python.tar.gz to /content
Files already downloaded and verified
Train Dataset: 40000
Valid Dataset: 10000
Test Dataset: 10000
Starting epoch 1/200, LR = 0.1
Current Avg Loss = 2.0584918291140823
Current Val Accuracy = 0.30063291139240506
Starting epoch 2/200, LR = 0.1
Current Avg Loss = 1.8754871074969952
Current Val Accuracy = 0.33860759493670883
Starting epoch 3/200, LR = 0.1
Current Avg Loss = 1.8070706098507612
Current Val Accuracy = 0.4017998417721519
Starting epoch 4/200, LR = 0.1
Current Avg Loss = 1.7620254907852564
Current Val Accuracy = 0.41356803797468356
Starting epoch 5/200, LR = 0.1
Current Avg Loss = 1.7282313811473358
Current Val Accuracy = 0.4464003164556962
Starting epoch 6/200, LR = 0.1
Current Avg Loss = 1.683744381635617
Current Val Accuracy = 0.44096123417721517
Starting epoch 7/200, LR = 0.1
Current Avg Loss = 1.669042733999399
Current Val Accuracy = 0.4639042721518987
Starting epoch 8/200, LR = 0.1
Current Avg Loss = 1.63616004356971

### Plot average loss and validation accuracy per epoch

In [ ]:
epochs = [i + 1 for i in range(NUM_EPOCHS)]
_, loss_snb, acc_snb = read_stats(FILE_PATH + '/shufflenetbig/stats.csv')
_, loss_sns, acc_sns = read_stats(FILE_PATH + '/shufflenetsmall/stats.csv')

filename = FILE_PATH + "/shufflenet_loss"
plot_stats(x=epochs, data=[loss_snb, loss_sns], save_path=filename, title="Loss per Epoch", x_label="Epoch", y_label="Loss", data_labels=["ShuffleNet-Big", "ShuffleNet-Small"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(loss_snb), max(loss_sns))))

acc_snb = [100*a for a in acc_snb]
acc_sns = [100*a for a in acc_sns]
filename = FILE_PATH + "/shufflenet_accuracy"
plot_stats(x=epochs, data=[acc_snb, acc_sns], save_path=filename, title="Validation Accuracy per Epoch", x_label="Epoch", y_label="Accuracy [%]", data_labels=["ShuffleNet-Big", "ShuffleNet-Small"], x_lim=(1, NUM_EPOCHS), y_lim=(0, max(max(acc_snb), max(acc_sns))))



In [ ]:
zip_name = "/content/fedmd/baseline_public_cifar10_3.zip"
!zip -r {zip_name} /content/fedmd/baseline_public_cifar10/
!cp {zip_name} /content/drive/MyDrive/understanding_federated_learning_project/baselines_results

  adding: content/fedmd/baseline_public_cifar10/ (stored 0%)
  adding: content/fedmd/baseline_public_cifar10/shufflenetv2/ (stored 0%)
  adding: content/fedmd/baseline_public_cifar10/shufflenetv2/shufflenetsmall/ (stored 0%)
  adding: content/fedmd/baseline_public_cifar10/shufflenetv2/shufflenetsmall/stats.csv (deflated 58%)
  adding: content/fedmd/baseline_public_cifar10/shufflenetv2/shufflenetsmall/best_model.pth (deflated 14%)
